# HW4 - Daniel Grossman

Importing our packages and setting working directory:

In [ ]:
import os
import pandas as pd
import re
os.chdir('/Users/danielgrossman/Documents/Coursework/BMI 312/Data/')

In [ ]:
files = os.listdir('/Users/danielgrossman/Documents/Coursework/BMI 312/Data/');
for f in files:
    if (f.endswith('.somatic.maf.short.gz')):
        #1) SOMATIC SHORT LOOP:
        #read in somatic short files
        somatic = pd.read_csv(f, sep='\t', compression='gzip', low_memory=False, comment='#')
        
        #parsing protein position + length
        parsed = [row.split("/") for row in somatic['prot.pos']]
        position, length = [x[0] for x in parsed], [x[1] for x in parsed]
        
        #adding protein info as columns
        somatic['protein_position'] = position
        somatic['protein_length'] = length
        
        #parsing allele top + bottom
        parsedaa = somatic['aa'].str.extract('(?P<reference_allele>\w+)/(?P<alternative_allele>\w+)')
        
        #joining parsed alleles as new columns
        somatic = somatic.join(parsedaa)
        
        
        #2) CLINICAL + SOMATIC MERGE:
        #setting pattern for re.findall
        idtype = '(^[\w\d]+\-[\w\d]+\-[\w\d]+)\-'
        
        #getting patient ID's, then adding as column
        patientID = [re.findall(idtype, s)[0] for s in somatic['sample_id']]
        somatic['patient_ID'] = patientID
        somatic['patient_ID'] = somatic['patient_ID'].astype(str)
        
    if (f.endswith('.clinical.short.txt')):
        clinical = pd.read_csv(f, sep='\t', index_col=0)
        clinical_somatic = clinical.join(somatic.set_index('patient'), on='patient', how='inner')
        clinical_somatic.shape[0]
        clinical_somatic.shape[1]
        clinical_somatic.to_csv(f, sep="\t", compression="gzip")